In [1]:
import numpy as np
import imageio
from skimage import transform
import matplotlib.pyplot as plt
from math import sqrt
import glob
import subprocess

%matplotlib inline

In [2]:
p = subprocess.Popen("ffmpeg -i data/videos/animals/2.mp4 -f wav -vn data/tmp/animals_0.wav", \
                     stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
p_status = p.wait()

Checking if there is an audio channel. If not, the output audio stream will be empty.

In [3]:
p = subprocess.Popen("ffprobe -i data/videos/animals/0.mp4 -show_streams -select_streams a -loglevel error"\
                     , stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
p_status = p.wait()

In [4]:
"No audio channel" if output.decode()=='' else "Audio channels present:"+output.decode()

'No audio channel'

In [5]:
!ffprobe -i data/videos/animals/0.mp4 -show_streams -select_streams a -loglevel error

In [6]:
p = subprocess.Popen("ffprobe -i /home/hemant/Videos/Titans\ -\ Season\ 1.mp4 -show_streams -select_streams a -loglevel error", \
                     stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
p_status = p.wait()

In [7]:
"No audio channel" if output.decode()=='' else "Audio channels present:"+output.decode()

'Audio channels present:[STREAM]\nindex=1\ncodec_name=aac\ncodec_long_name=AAC (Advanced Audio Coding)\nprofile=LC\ncodec_type=audio\ncodec_time_base=1/44100\ncodec_tag_string=mp4a\ncodec_tag=0x6134706d\nsample_fmt=fltp\nsample_rate=44100\nchannels=2\nchannel_layout=stereo\nbits_per_sample=0\nid=N/A\nr_frame_rate=0/0\navg_frame_rate=0/0\ntime_base=1/44100\nstart_pts=0\nstart_time=0.000000\nduration_ts=114762752\nduration=2602.329977\nbit_rate=125588\nmax_bit_rate=N/A\nbits_per_raw_sample=N/A\nnb_frames=112073\nnb_read_frames=N/A\nnb_read_packets=N/A\nDISPOSITION:default=1\nDISPOSITION:dub=0\nDISPOSITION:original=0\nDISPOSITION:comment=0\nDISPOSITION:lyrics=0\nDISPOSITION:karaoke=0\nDISPOSITION:forced=0\nDISPOSITION:hearing_impaired=0\nDISPOSITION:visual_impaired=0\nDISPOSITION:clean_effects=0\nDISPOSITION:attached_pic=0\nDISPOSITION:timed_thumbnails=0\nTAG:creation_time=2018-12-03T23:09:27.000000Z\nTAG:language=eng\nTAG:handler_name=ISO Media file produced by Google Inc. Created on: 12

In [8]:
p = subprocess.Popen("ffmpeg -i /home/hemant/Videos/Titans\ -\ Season\ 1.mp4 -f wav -vn data/tmp/animals_0.wav", \
                     stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
p_status = p.wait()

In [9]:
# The file is huge. Taking a small chunk of it and delete the larger chunk
!ffmpeg -t 30 -i data/tmp/animals_0.wav data/tmp/mini_0.wav -loglevel error && rm animals_0.wav

In [10]:
# sample script to create a log_mel_spectrogram
from scipy.io import wavfile
from scipy import signal

sample_rate=16000
window_size=20
step_size=10
eps=1e-10
rate, data = wavfile.read('data/tmp/mini_0.wav')
if data.ndim > 1 : # ignore  channels 2+
    data = data[:, 0]
nperseg = int(round(window_size * sample_rate / 1e3))
noverlap = int(round(step_size * sample_rate / 1e3))
freqs, times, spec = signal.spectrogram(data,fs=sample_rate,window='hann',nperseg=nperseg,noverlap=noverlap)
log_specgram = np.log(spec.T.astype(np.float32) + eps)

In [11]:
print(log_specgram.shape)

(8267, 161)


In [12]:
import sys
sys.path.append("audioset/")

The module **vggish_input** has a function **wavfile_to_examples** which returns the log_mel_spectrogram (after correction intro standard form) of the input **wav_file**.

In [13]:
import vggish_input
features_tensor = vggish_input.wavfile_to_examples(wav_file='data/tmp/mini_0.wav')

827 ms ± 20 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [14]:
features_tensor.shape

(31, 96, 64)

The module **vggish_inference** has a function **main** which returns the vggish extracted embedding (from the log_mel_spectrogram that is in turn created by vggish_input) of the input **wav_file**.

In [15]:
import vggish_inference
embedding_batch, postprocessed_batch = vggish_inference.main(wav_file='data/tmp/mini_0.wav')

INFO:tensorflow:Restoring parameters from audioset/vggish_model.ckpt


1.94 s ± 6.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [16]:
embedding_batch.shape

(31, 128)

Here $31$ is the time in seconds of the audio stream and $128$ is the dimensionality of the extracted vggish embedding.